# 작물별 생육 기준 데이터 생성
작물의 생육에 필요한 온도, 습도, 조도, 토양 수분 범위를 설정하고,
조도(lux)를 ADC 값으로 변환하여 범위를 계산합니다.

In [ ]:
# ==== 필요한 라이브러리 불러오기 ====
import pandas as pd                   # 데이터프레임(표 형식)으로 데이터 다루기
from sqlalchemy import create_engine  # 데이터베이스에 연결하기 위한 엔진 생성용
import pymysql                        # MariaDB(MySQL) 연결용

# 1. ==== 작물별 기준값 정의 + lux → ADC 변환 ====

# 작물별로 필요한 환경 조건을 정의합니다.
# 조도는 lux 단위이지만, 우리가 센서로 측정하는 값은 adc 단위이기 때문에 변환이 필요해요.
data = {
    '작물': ['상추', '바질', '민트', '고추', '딸기', '토마토', '시금치', '로즈마리', '산호수', '몬스테라'],
    'temp_min': [15, 21, 18, 23, 19, 21, 13, 19, 17, 22],    # 최소 온도
    'temp_max': [21, 27, 24, 29, 25, 27, 19, 25, 23, 28],    # 최대 온도
    'humid_min': [55, 60, 55, 50, 65, 55, 55, 40, 60, 60],   # 최소 습도
    'humid_max': [75, 80, 75, 70, 85, 75, 75, 60, 80, 80],   # 최대 습도
    'soil_min': [20, 30, 30, 25, 40, 25, 30, 15, 25, 30],    # 최소 토양 수분 (%)
    'soil_max': [30, 40, 40, 35, 50, 35, 40, 25, 35, 40],    # 최대 토양 수분 (%)
    'lux_min': [2500, 10000, 3000, 20000, 15000, 20000, 1000, 7000, 1000, 1000],  # 최소 조도 (lux)
    'lux_max': [12500, 20000, 13000, 30000, 25000, 30000, 11000, 17000, 5000, 11000],  # 최대 조도 (lux)
}

# 위 데이터를 데이터프레임(표)로 만듭니다.
df = pd.DataFrame(data)

# lux → adc 단위로 변환하는 함수 정의
# 조도 센서는 lux 값이 낮을수록 adc 값이 높습니다 (반비례)
def lux_to_adc(lux):
    adc = 1023 - (lux / 30000 * 1023)
    return round(adc)

# lux_max는 adc_min으로 변환됩니다 (밝은 쪽 기준)
df['adc_min'] = df['lux_max'].apply(lux_to_adc)

# lux_min은 adc_max로 변환됩니다 (어두운 쪽 기준)
df['adc_max'] = df['lux_min'].apply(lux_to_adc)

# 각 작물에 고유한 ID 부여 (Primary Key 용도)
df.insert(0, 'id', range(1, len(df) + 1))

# 2. ==== MariaDB 연결 설정 ====
# HOST에 팀원들 IP를 입력하면 자동으로 저장된다. 
user = 'team4_user'
password = 'mariadb'
host = '192.168.30.91'
port = 3306
database = 'team4'

# SQLAlchemy를 이용해 데이터베이스 엔진 생성
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}")

# 3. ==== crop_standards 테이블 생성 ====
# 기존에 같은 이름의 테이블이 있으면 덮어씌움 (replace)
df.to_sql(name='crop_standards', con=engine, if_exists='replace', index=False)

# 4. ==== id 컬럼을 기본키(PK) + 자동 증가 설정 ====
conn = pymysql.connect(host=host, user=user, password=password, database=database, port=port)
cursor = conn.cursor()

cursor.execute("""
    ALTER TABLE crop_standards
    MODIFY COLUMN id INT NOT NULL AUTO_INCREMENT,
    ADD PRIMARY KEY (id);
""")

# DB 변경사항 저장 및 연결 종료
conn.commit()
cursor.close()
conn.close()

# 최종 데이터프레임 출력 (확인용)
df


# 단계	설명
# ① 데이터 준비	작물별 온도, 습도, 수분, 조도(lux) 기준값을 테이블로 정리
# ② 단위 변환	조도(lux)를 ADC 값으로 변환 (센서 기준에 맞게)
# ③ DB 연결	MariaDB에 연결해 테이블 저장
# ④ 테이블 설정	id를 Primary Key + Auto Increment 설정


,id,작물,temp_min,temp_max,humid_min,humid_max,soil_min,soil_max,lux_min,lux_max,adc_min,adc_max
0,1,상추,15,21,55,75,20,30,2500,12500,597,938
1,2,바질,21,27,60,80,30,40,10000,20000,341,682
2,3,민트,18,24,55,75,30,40,3000,13000,580,921
3,4,고추,23,29,50,70,25,35,20000,30000,0,341
4,5,딸기,19,25,65,85,40,50,15000,25000,170,512
5,6,토마토,21,27,55,75,25,35,20000,30000,0,341
6,7,시금치,13,19,55,75,30,40,1000,11000,648,989
7,8,로즈마리,19,25,40,60,15,25,7000,17000,443,784
8,9,산호수,17,23,60,80,25,35,1000,5000,852,989
9,10,몬스테라,22,28,60,80,30,40,1000,11000,648,989
